### Tweet Pre-Processing:
Tweets consist of many acronyms, emoticons and unnecessary data like pictures and URL’s. So, tweets are pre-processed to represent correct emotions of public. 

In [1]:
#libraries
import numpy as np
import pandas as pd
import re
import datetime
import csv

In [2]:
#read csv file
tweets = pd.read_csv("C:/Users/user/Downloads/tweets.csv")

In [29]:
tweets.head()

,Date,Text,time,year,month,seconds
0,2018-03-21 16:59:36,b'A Thought on Apple Inc. iPhone X Plus Pricin...,16:59:36,2018,03,36
1,2018-03-21 14:09:40,b'#Options #OptionsTrading #AAPL #StockMarket...,14:09:40,2018,03,40
2,2018-03-21 13:51:23,b'#Options #OptionsTrading #AAPL #StockMarket...,13:51:23,2018,03,23
3,2018-03-21 13:38:42,b'#Options #OptionsTrading #AAPL #StockMarket...,13:38:42,2018,03,42
4,2018-03-21 13:31:15,"b'https://t.co/qkrb3Kese5 ""#Apple #AAPL now re...",13:31:15,2018,03,15


In [4]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1370 entries, 0 to 1369
Data columns (total 2 columns):
Date    1370 non-null object
Text    1370 non-null object
dtypes: object(2)
memory usage: 21.5+ KB


In [5]:
tweets['Date'] = pd.to_datetime(tweets['Date'])

In [7]:
tweets['time'] = tweets['Date'].dt.strftime('%H:%M:%S')

In [8]:
tweets['year'] = tweets['Date'].dt.strftime('%Y')

In [9]:
tweets['month'] = tweets['Date'].dt.strftime('%m')

In [10]:
tweets['seconds'] = tweets['Date'].dt.strftime('%S')

In [30]:
tweets.head(10)

,Date,Text,time,year,month,seconds
0,2018-03-21 16:59:36,b'A Thought on Apple Inc. iPhone X Plus Pricin...,16:59:36,2018,03,36
1,2018-03-21 14:09:40,b'#Options #OptionsTrading #AAPL #StockMarket...,14:09:40,2018,03,40
2,2018-03-21 13:51:23,b'#Options #OptionsTrading #AAPL #StockMarket...,13:51:23,2018,03,23
3,2018-03-21 13:38:42,b'#Options #OptionsTrading #AAPL #StockMarket...,13:38:42,2018,03,42
4,2018-03-21 13:31:15,"b'https://t.co/qkrb3Kese5 ""#Apple #AAPL now re...",13:31:15,2018,03,15
5,2018-03-21 13:25:02,b'\xe2\x9d\x97\xef\xb8\x8f Only 5 minutes unti...,13:25:02,2018,03,02
6,2018-03-21 13:13:59,b'RT @JPDesloges: #Apple : \nApple shares Brok...,13:13:59,2018,03,59
7,2018-03-21 13:05:00,b'#Apple : \nApple shares Broke on March 16 th...,13:05:00,2018,03,00
8,2018-03-21 07:42:01,b'RT @AmanKabirMPS: Dow Jones: on the way down...,07:42:01,2018,03,01
9,2018-03-21 07:07:03,b'Apple Inc (AAPL) price at close for today is...,07:07:03,2018,03,03


In [12]:
tweet = tweets['Text']

In [13]:
tweet.to_csv('tweet.csv')

### Preprocess Tweets:
Preprocessing includes converting everything to lowercase, converting http or www to URL, usernames to AT_USER, adding white spaces and removing hastags.

In [14]:
#start process_tweet
def processTweet(tweet):
    # process the tweets
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet
#end

#Read the tweets one by one and process it
fp = open('tweet.csv', 'r')
line = fp.readline()

while line:
    processedTweet = processTweet(line)
    print(processedTweet)
    line = fp.readline()
#end loop
fp.close()

0,b'a thought on apple inc. iphone x plus pricing yahoo: URL aapl.us united states s&amp;p 500 inde\xe2\x80\xa6 URL 
1,b'options optionstrading aapl stockmarket trading \n\nbto $aapl mar23 175 call AT_USER filled 10:07am' 
2,b'options optionstrading aapl stockmarket trading \n\nstc $aapl mar23 175 call AT_USER filled 9:50am' 
3,b'options optionstrading aapl stockmarket trading \n\nbto $aapl mar23 175 call AT_USER filled 9:35am' 
4,"b'URL ""apple aapl now refocusing on strengthening its footprint in education technology\xe2\x80\xa6 URL 
5,"b'\xe2\x9d\x97\xef\xb8\x8f only 5 minutes until the openingbell \xe2\x9d\x97\xef\xb8\x8f \njoin our stock, options, futures, and crypto\nchatrooms for real-ti\xe2\x80\xa6 URL 
6,b'rt AT_USER apple : \napple shares broke on march 16 the daily uptrend channel support trendline.+ still into a daily downtrend channel\xe2\x80\xa6' 
7,b'apple : \napple shares broke on march 16 the daily uptrend channel support trendline.+ still into a daily downtrend\xe2\x8

### Removing stopwords, punctuations,repeated letters, etc.

In [15]:
stopwords = []
#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end

#start getStopWordList
def getStopWordList(stopWordsList):
    #read the stopwords file and build a list
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open('stopWordsList.txt', 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords
#end

### Feature Vector:

In [16]:
#start getfeatureVector
def getFeatureVector(tweet):
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector
#end

In [17]:
#Read the tweets one by one and process it
fp = open('tweet.csv', 'r')
line = fp.readline()

st = open('stopWordsList.txt', 'r')
stopWords = getStopWordList('stopWordsList.txt')

while line:
    processedTweet = processTweet(line)
    featureVector = getFeatureVector(processedTweet)
    print(featureVector)
    line = fp.readline()
#end loop
fp.close()

['thought', 'on', 'apple', 'inc', 'iphone', 'x', 'plus', 'pricing', 'united', 'states']
['optionstrading', 'aapl', 'stockmarket', 'trading', 'mar23', 'call', 'filled']
['optionstrading', 'aapl', 'stockmarket', 'trading', 'mar23', 'call', 'filled']
['optionstrading', 'aapl', 'stockmarket', 'trading', 'mar23', 'call', 'filled']
['apple', 'aapl', 'now', 'refocusing', 'on', 'strengthening', 'its', 'footprint', 'in', 'education']
['only', 'minutes', 'until', 'the', 'openingbell', 'our', 'stock', 'options', 'futures', 'and', 'for']
['apple', 'shares', 'broke', 'on', 'march', 'the', 'daily', 'uptrend', 'channel', 'support', 'still', 'into', 'a', 'daily', 'downtrend']
['shares', 'broke', 'on', 'march', 'the', 'daily', 'uptrend', 'channel', 'support', 'still', 'into', 'a', 'daily']
['dow', 'on', 'the', 'way', 'down', 'look', 'for', 'strong', 'levels', 'at', 'and']
['inc', 'price', 'at', 'close', 'for', 'today', 'is', 'apple', 'aapl', 'finance', 'investments', 'roi']
['in', 'stocks', 'interested

['optionstrading', 'aapl', 'stockmarket', 'trading', 'mar16', 'call', 'filled', 'reach', 'my', 'stop', 'limit']
['optionstrading', 'aapl', 'stockmarket', 'trading', 'limit', 'mar16', 'call']
['optionstrading', 'aapl', 'stockmarket', 'trading', 'mar16', 'call', 'filled']
['someone', 'out', 'there', 'in', 'constipation', 'land', 'needs', 'a', 'new', 'why', 'else', 'aapl']
['only', 'minutes', 'until', 'the', 'openingbell', 'our', 'stock', 'options', 'futures', 'and', 'for']
['only', 'minutes', 'until', 'the', 'openingbell', 'our', 'stock', 'options', 'futures', 'and', 'for']
['market', 'in', 'ford', 'upgrade', 'google', 'to', 'ban', 'crypto', 'ads', 'bezos', 'and', 'amazon']
['apple', 'shares', 'tested', 'and', 'rejected', 'on', 'march', 'the', 'major', 'daily', 'resistance', 'trendline', 'yellow', 'on', 'low']
['apple', 'shares', 'tested', 'and', 'rejected', 'on', 'march', 'the', 'major', 'daily', 'resistance', 'trendline', 'yellow', 'on', 'low']
['shares', 'tested', 'and', 'rejected', '

In [18]:
#Read the tweets one by one and process it
inpTweets = csv.reader(open('tweet.csv', 'r'), delimiter=',', quotechar='|')
stopWords = getStopWordList('stopWordsList.txt')
featureList = []

for row in inpTweets:
    tweet = row[1]
    processedTweet = processTweet(tweet)
    featureVector = getFeatureVector(processedTweet)
    featureList.append((featureVector));
#end loop

In [19]:
featureList

[['thought',
  'on',
  'apple',
  'inc',
  'iphone',
  'x',
  'plus',
  'pricing',
  'united',
  'states'],
 ['optionstrading',
  'aapl',
  'stockmarket',
  'trading',
  'mar23',
  'call',
  'filled'],
 ['optionstrading',
  'aapl',
  'stockmarket',
  'trading',
  'mar23',
  'call',
  'filled'],
 ['optionstrading',
  'aapl',
  'stockmarket',
  'trading',
  'mar23',
  'call',
  'filled'],
 ['apple',
  'aapl',
  'now',
  'refocusing',
  'on',
  'strengthening',
  'its',
  'footprint',
  'in',
  'education'],
 ['only', 'minutes', 'until', 'the', 'openingbell', 'our', 'stock'],
 ['apple',
  'shares',
  'broke',
  'on',
  'march',
  'the',
  'daily',
  'uptrend',
  'channel',
  'support',
  'still',
  'into',
  'a',
  'daily',
  'downtrend'],
 ['shares',
  'broke',
  'on',
  'march',
  'the',
  'daily',
  'uptrend',
  'channel',
  'support',
  'still',
  'into',
  'a',
  'daily'],
 ['dow', 'on', 'the', 'way', 'down'],
 ['inc',
  'price',
  'at',
  'close',
  'for',
  'today',
  'is',
  'appl

In [20]:
from functools import reduce
import operator
final_feature_list = reduce(operator.concat, featureList)

In [21]:
final_feature_list

['thought',
 'on',
 'apple',
 'inc',
 'iphone',
 'x',
 'plus',
 'pricing',
 'united',
 'states',
 'optionstrading',
 'aapl',
 'stockmarket',
 'trading',
 'mar23',
 'call',
 'filled',
 'optionstrading',
 'aapl',
 'stockmarket',
 'trading',
 'mar23',
 'call',
 'filled',
 'optionstrading',
 'aapl',
 'stockmarket',
 'trading',
 'mar23',
 'call',
 'filled',
 'apple',
 'aapl',
 'now',
 'refocusing',
 'on',
 'strengthening',
 'its',
 'footprint',
 'in',
 'education',
 'only',
 'minutes',
 'until',
 'the',
 'openingbell',
 'our',
 'stock',
 'apple',
 'shares',
 'broke',
 'on',
 'march',
 'the',
 'daily',
 'uptrend',
 'channel',
 'support',
 'still',
 'into',
 'a',
 'daily',
 'downtrend',
 'shares',
 'broke',
 'on',
 'march',
 'the',
 'daily',
 'uptrend',
 'channel',
 'support',
 'still',
 'into',
 'a',
 'daily',
 'dow',
 'on',
 'the',
 'way',
 'down',
 'inc',
 'price',
 'at',
 'close',
 'for',
 'today',
 'is',
 'apple',
 'aapl',
 'finance',
 'investments',
 'roi',
 'in',
 'stocks',
 'intereste

In [22]:
#remove duplicates from list
final_feature_list = list(set(final_feature_list))
final_feature_list

['take',
 'memoir',
 'am',
 'kingdom',
 'trade',
 'from',
 'cap',
 'itsoftware',
 'report',
 'alhubeco',
 'finance',
 'needs',
 'stores',
 'relevant',
 'see',
 'more',
 'cheat',
 'tech',
 'with',
 'compression',
 'bitcoin',
 'worth',
 'spx',
 'jnewsome',
 'momentum',
 'equal',
 'waiting',
 'just',
 'currently',
 'education',
 'look',
 'breakout',
 'spotify',
 'decisive',
 'expected',
 'pattern',
 'price',
 'major',
 'techcombank',
 'monday',
 'stockstowatch',
 'screens',
 'suspends',
 'wednesday',
 'know',
 'updated',
 'series',
 'uptrend',
 'tested',
 'strengthening',
 'movies',
 'crash',
 'disney',
 'ftse',
 'strong',
 'bonds',
 'djia',
 'lifestyle',
 'breaking',
 'growing',
 'takes',
 'europe',
 'sweetens',
 'thrice',
 'pro',
 'fix',
 'our',
 'notifications',
 'silenceisgolden',
 'models',
 'plan',
 'set',
 'contributors',
 'walk',
 'are',
 'stupid',
 'trips',
 'employees',
 'revenue',
 'was',
 'childhood',
 'oled',
 'anticipating',
 'target',
 'live',
 'her',
 'probably',
 'acquire

In [23]:
#start extract_features
def extract_features(featureVector):
    tweet_words = set(featureVector)
    features = {}
    for word in final_feature_list:
        features['contains(%s)' % word] = (word in tweet_words)
    return features
#end

In [24]:
import nltk
training_set = nltk.classify.util.apply_features(extract_features, featureList)

### Naive Bayes Classifier:
Naïve Bayes classifier is a supervised classification algorithm. It is based on “Bayes” theorem with a “naïve” assumption of independence between every pair of feature in feature vector. Naïve Bayes helps us to classify tweets using Tweepy into three categories: positive, negative and neutral.

In [25]:
# Train the classifier
NBClassifier = nltk.NaiveBayesClassifier.train(training_set)

In [26]:
def get_tweet_sentiment(tweets):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(processTweet(tweets))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

In [28]:
# print informative features about the classifier
print(NBClassifier.show_most_informative_features(10))

Most Informative Features
             contains(i) = False              on : aapl   =      2.8 : 1.0
             contains(a) = True             aapl : shares =      1.2 : 1.0
             contains(a) = False              on : minute =      1.1 : 1.0
             contains(i) = True              the : aapl   =      1.1 : 1.0
           contains(gap) = False             the : on     =      1.0 : 1.0
          contains(kept) = False             the : on     =      1.0 : 1.0
       contains(support) = False             the : on     =      1.0 : 1.0
         contains(group) = False             the : on     =      1.0 : 1.0
         contains(congo) = False             the : on     =      1.0 : 1.0
           contains(our) = False             the : on     =      1.0 : 1.0
None


#### Reference:
https://www.ravikiranj.net/posts/2012/code/how-build-twitter-sentiment-analyzer/#filtering-tweet-words-for-feature-vector